In [1]:
import numpy as np
from os.path import exists
from os import listdir
from os.path import isfile, join
from datetime import datetime

# return a list of budget for each gram
def evenly_distribute_first_half(gram_num, gram_output_num_list, remaining_budget):
    
    n = len(gram_output_num_list)
    budget_allocation = np.zeros((n,), dtype = "int")
    
    # remember the invalid index so that we can ignore it in the next allocation
    invaid_index = {}
    
    while remaining_budget != 0 and len(invaid_index) != len(gram_output_num_list):
        iterate_index_list = []
        for i in range(n):
            if i not in invaid_index:
                iterate_index_list.append(i)
        #remaining_budget = budget - avg * gram_num
        '''print("************************")
        print(budget_allocation)
        print(invaid_index)
        print(remaining_budget)
        print(iterate_index_list)
        print("************************")'''
        avg = int(remaining_budget / len(iterate_index_list))
        # enough to allocate to every one
        if avg > 0:
            for index in iterate_index_list:
                if gram_output_num_list[index] - budget_allocation[index] < avg:
                    # this one is full now
                    remaining_budget -= gram_output_num_list[index] - budget_allocation[index]
                    budget_allocation[index] = gram_output_num_list[index]
                    invaid_index[index] = True
                else:
                    budget_allocation[index] += avg
                    remaining_budget -= avg
        # only can allocate once
        else:
            for index in iterate_index_list:
                if remaining_budget == 0:
                    break
                else:
                    if gram_output_num_list[index] - budget_allocation[index] < 1:
                        invaid_index[index] = True
                    else:
                        budget_allocation[index] += 1
                        remaining_budget -= 1
                    
    return budget_allocation


def evenly_distribute(gram_num, gram_output_num_list, remaining_budget):
    
    n = len(gram_output_num_list)
    budget_allocation = np.zeros((n,), dtype = "int")
    
    # remember the invalid index so that we can ignore it in the next allocation
    invaid_index = {}
    
    while remaining_budget != 0 and len(invaid_index) != len(gram_output_num_list):
        iterate_index_list = []
        for i in range(n):
            if i not in invaid_index:
                iterate_index_list.append(i)
        #remaining_budget = budget - avg * gram_num
        '''print("************************")
        print(budget_allocation)
        print(invaid_index)
        print(remaining_budget)
        print(iterate_index_list)
        print("************************")'''
        avg = int(remaining_budget / len(iterate_index_list))
        # enough to allocate to every one
        if avg > 0:
            for index in iterate_index_list:
                if gram_output_num_list[index] - budget_allocation[index] < avg:
                    # this one is full now
                    remaining_budget -= gram_output_num_list[index] - budget_allocation[index]
                    budget_allocation[index] = gram_output_num_list[index]
                    invaid_index[index] = True
                else:
                    budget_allocation[index] += avg
                    remaining_budget -= avg
        # only can allocate once
        else:
            for index in iterate_index_list:
                if remaining_budget == 0:
                    break
                else:
                    if gram_output_num_list[index] - budget_allocation[index] < 1:
                        invaid_index[index] = True
                    else:
                        budget_allocation[index] += 1
                        remaining_budget -= 1
                    
    return budget_allocation, remaining_budget


def compute_hit_ratio_longer_gram_half_prior(query_file_name, budget, top_k):
    # use an axiliary dictionary to store the allocated budget for each gram file
    line_num_allocated = {}
    reserved_budget = int(budget / 2)
    budget = budget - reserved_budget
    
    underline_pos = query_file_name.index("_")
    query_index = query_file_name[:underline_pos]
    query =  query_file_name[underline_pos + 1:]
    
    query_term_list = query.split()
    length = len(query_term_list)
    
    single_list = []
    duplet_list = []
    
    for i in range(length):
        single_list.append(query_term_list[i])
        
    if length >= 2:
        for i in range(length):
            for j in range(i + 1, length):
                duplet_list.append(query_term_list[i] + " " + query_term_list[j])
                
    
    # determine which file occur in our first layer index
    # start from quadruplet
    valid_gram_num = 0
    gram_output_num_list = []
    gram_filename_list = []
            
    for duplet in duplet_list:
        filename = "DUPLET_" + duplet
        
        if exists("./First Layer Index/duplet/" + filename):
            valid_gram_num += 1
            temp_file = open("./First Layer Index/duplet/" + filename, "r", encoding='utf-8')
            output_count = 0
            temp_line = temp_file.readline()
            while temp_line:
                output_count += 1
                temp_line = temp_file.readline()
            #print(output_count)
            gram_output_num_list.append(output_count)
            gram_filename_list.append("./First Layer Index/duplet/" + filename)
            temp_file.close()
            
    for single in single_list:
        filename = "SINGLE_" + single
        
        if exists("./First Layer Index/single/" + filename):
            valid_gram_num += 1
            temp_file = open("./First Layer Index/single/" + filename, "r", encoding='utf-8')
            output_count = 0
            temp_line = temp_file.readline()
            while temp_line:
                output_count += 1
                temp_line = temp_file.readline()
            #print(output_count)
            gram_output_num_list.append(output_count)
            gram_filename_list.append("./First Layer Index/single/" + filename)
            temp_file.close()
            
    #print("Sanity Check:", valid_gram_num == len(gram_output_num_list))
    
    budget_allocation_list = evenly_distribute_first_half(valid_gram_num, gram_output_num_list, budget)
    
    #print("Budget:", budget)
    #print("Reserved Budget:", reserved_budget)
    #print(budget_allocation_list)
    
    # iterate over the allocation list to get the remaining available results num for each gram file
    for i in range(len(gram_filename_list)):
        line_num_allocated[gram_filename_list[i]] = budget_allocation_list[i]

    # determine which file occur in our first layer index
    # start from quadruplet
    # gram_filename_list = []
    
    budget = reserved_budget
    
    valid_gram_num_duplet = 0
    gram_output_num_list_duplet = []
    gram_filename_list_duplet = []
    
    for duplet in duplet_list:
        filename = "DUPLET_" + duplet
        
        if exists("./First Layer Index/duplet/" + filename):
            valid_gram_num_duplet += 1
            temp_file = open("./First Layer Index/duplet/" + filename, "r", encoding='utf-8')
            output_count = 0
            temp_line = temp_file.readline()
            while temp_line:
                output_count += 1
                temp_line = temp_file.readline()
            #print(output_count)
            temp_file.close()
            gram_output_num_list_duplet.append(output_count - line_num_allocated["./First Layer Index/duplet/" + filename])
            gram_filename_list_duplet.append("./First Layer Index/duplet/" + filename)
            #gram_filename_list.append("./First Layer Index/duplet/" + filename)
            
    valid_gram_num_single = 0
    gram_output_num_list_single = []
    gram_filename_list_single = []
    
    for single in single_list:
        filename = "SINGLE_" + single
        
        if exists("./First Layer Index/single/" + filename):
            valid_gram_num_single += 1
            temp_file = open("./First Layer Index/single/" + filename, "r", encoding='utf-8')
            output_count = 0
            temp_line = temp_file.readline()
            while temp_line:
                output_count += 1
                temp_line = temp_file.readline()
            #print(output_count)
            temp_file.close()
            gram_output_num_list_single.append(output_count - line_num_allocated["./First Layer Index/single/" + filename])
            gram_filename_list_single.append("./First Layer Index/single/" + filename)
            #gram_filename_list.append("./First Layer Index/single/" + filename)
            
            
    #print("Sanity Check:", valid_gram_num == len(gram_output_num_list))
    
    budget_allocation_list_duplet, budget = evenly_distribute(valid_gram_num_duplet, gram_output_num_list_duplet, budget)
    budget_allocation_list_single, budget = evenly_distribute(valid_gram_num_single, gram_output_num_list_single, budget)
    #budget_allocation_list = budget_allocation_list_quadruplet + budget_allocation_list_triplet + budget_allocation_list_duplet + budget_allocation_list_single
    temp_budget_allocation_list = []

    for allocation_duplet in budget_allocation_list_duplet:
        temp_budget_allocation_list.append(allocation_duplet) 
    for allocation_single in budget_allocation_list_single:
        temp_budget_allocation_list.append(allocation_single) 
    
    #print(budget_allocation_list_quadruplet, budget_allocation_list_triplet, budget_allocation_list_duplet, budget_allocation_list_single, budget)
    #print(budget_allocation_list_quadruplet)
    #print(gram_filename_list)
    
    for i in range(len(budget_allocation_list)):
        budget_allocation_list[i] += temp_budget_allocation_list[i]
    
    predict_did_list = set()
    
    for index in range(len(gram_filename_list)):
        temp_file = open(gram_filename_list[index], "r", encoding="utf-8")
        temp_budget = budget_allocation_list[index]
        for i in range(temp_budget):
            temp_line = temp_file.readline()
            temp_line_list = temp_line.replace("\n", "").split()
            did = temp_line_list[1]
            predict_did_list.add(did)
        temp_file.close()
        
    #print("Compute for predict completed")
    
    real_did_list = set()
    real_output_file = open("./Split Test Query Output/" + query_file_name, "r", encoding='utf-8')
    real_output_line = real_output_file.readline()
    top_count = 0
    while real_output_line and top_count < top_k:
        line_list = real_output_line.replace("\n", "").split()
        did = line_list[1]
        real_did_list.add(did)
        top_count += 1
        real_output_line = real_output_file.readline()
    
    real_output_file.close()
    
    # compute accuracy:
    intersection_list = sorted(list(predict_did_list & real_did_list))
    real_list = sorted(list(real_did_list))
    '''
    print("*********************************")
    print("Budget is:", budget)
    print("The accuracy is:" + str(len(intersection_list)) + "/" + str(len(real_list)) + " = " + str(len(intersection_list) / len(real_list)))
    print("*********************************")
    print()
    '''
    if len(real_list) == 0:
        return None
    return len(intersection_list) / len(real_list)

In [2]:
def compute_system_accuracy(directory_name, budget, top_k):
    onlyfiles = [f for f in listdir(directory_name) if isfile(join(directory_name, f))]
    #print(len(onlyfiles))
    
    accuracy_list = []
    #count = 0
    for test_query_filename in onlyfiles:
        #count += 1
        current_acc = compute_hit_ratio_longer_gram_half_prior(test_query_filename, budget, top_k)
        if current_acc != None:
            accuracy_list.append(current_acc)
        #if count % 1000 == 0:
        #    print(datetime.now())
        
    print("*********************************")
    print("Budget is:", budget)
    print("The average accuracy is:", sum(accuracy_list) / len(accuracy_list))
    print("*********************************")
    print()

In [3]:
start_time = datetime.now()
top_k = 10
acc = compute_hit_ratio_longer_gram_half_prior("9759003_drowings inuyasha", 4000, top_k)
print(acc)
total_running_time = datetime.now() - start_time
print("Time to Compute Accuracy:", str(total_running_time.total_seconds()) + "s")

0.7
Time to Compute Accuracy: 0.025672s


In [4]:
start_time = datetime.now()
top_k = 10
compute_system_accuracy("./Split Test Query Output", 2000, top_k)
total_running_time = datetime.now() - start_time
print("Time to Compute Average Accuracy:", str(total_running_time.total_seconds()) + "s")

*********************************
Budget is: 2000
The average accuracy is: 0.8862205168716826
*********************************

Time to Compute Average Accuracy: 420.260456s


In [5]:
budget_list = [500, 1000, 2000, 3000, 4000, 6000, 8000, 12000, 15000, 18000, 25000, 35000, 50000, 5000]
top_k = 10
start_time = datetime.now()
for budget in budget_list:
    compute_system_accuracy("./Split Test Query Output", budget, top_k)
total_running_time = datetime.now() - start_time
print("Time to Compute Average Accuracy for different budget:", str(total_running_time.total_seconds()) + "s")

*********************************
Budget is: 500
The average accuracy is: 0.7773953278410997
*********************************

*********************************
Budget is: 1000
The average accuracy is: 0.8368919986457488
*********************************

*********************************
Budget is: 2000
The average accuracy is: 0.8862205168716826
*********************************

*********************************
Budget is: 3000
The average accuracy is: 0.9116126847985533
*********************************

*********************************
Budget is: 4000
The average accuracy is: 0.9268592709626418
*********************************

*********************************
Budget is: 6000
The average accuracy is: 0.9455253357408837
*********************************

*********************************
Budget is: 8000
The average accuracy is: 0.9567543166685456
*********************************

*********************************
Budget is: 12000
The average accuracy is: 0.9673851709739288
***